In [1]:
!pip3 install transformers torch


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch


/Users/shreyasaniya/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/shreyasaniya/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load pre-trained model and tokenizer
model_name = "gpt2"  # You can replace with 'EleutherAI/gpt-neo-125M' for GPT-Neo
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [8]:
def generate_story(prompt, model, tokenizer, device, max_length=500):
    # Encode the prompt text into tokens
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    # Create an attention mask
    attention_mask = torch.ones(inputs.shape, device=device)
    
    # Generate a story with the attention mask
    outputs = model.generate(
        inputs, 
        max_length=max_length,  # Maximum length of the generated story
        num_return_sequences=1,  # Number of stories to generate
        no_repeat_ngram_size=2,  # Prevent repeating phrases
        do_sample=True,  # Use sampling to increase creativity
        top_k=50,  # Sampling: Consider top 50 words
        top_p=0.95,  # Sampling: Consider top probability
        temperature=0.7,  # Creativity control
        pad_token_id=tokenizer.eos_token_id,  # End of sequence token
        attention_mask=attention_mask  # Pass the attention mask here
    )
    
    # Decode the generated story from tokens to text
    story = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return story


In [15]:
import re
import textwrap

def clean_story(story, max_line_length=80):
    # Remove unwanted line breaks and extra spaces
    story = story.replace("\n", " ").replace("  ", " ").strip()

    # Split the story into sentences
    sentences = re.split(r'(?<=[.!?]) +', story)

    # Format the sentences into paragraphs, where each paragraph contains 3-5 sentences
    paragraphs = []
    paragraph = []
    for sentence in sentences:
        paragraph.append(sentence)
        
        # Add paragraph after 3-5 sentences
        if len(paragraph) >= 3:  # You can change this number for more/less sentences per paragraph
            paragraphs.append(" ".join(paragraph))
            paragraph = []
    
    # If there are remaining sentences, add them as the last paragraph
    if paragraph:
        paragraphs.append(" ".join(paragraph))

    # Join the paragraphs with a blank line between them
    formatted_story = "\n\n".join(paragraphs)

    # Wrap each paragraph to a max line length for readability
    wrapped_story = "\n\n".join([textwrap.fill(paragraph, width=max_line_length) for paragraph in paragraphs])

    return wrapped_story


In [17]:
# Define the prompt to start the story
prompt = "Once upon a time, in a magical forest, a young prince discovered a hidden treasure..."

# Generate the story
story = generate_story(prompt, model, tokenizer, device)

# Print the generated story
print(story)


Once upon a time, in a magical forest, a young prince discovered a hidden treasure...


He is a prince who is in need of a new name.

—Dwarves of the Forest, The Dungeon World, D&D 2nd Edition, pp. 12-13

. (See the "A" in the bottom of this article)

, and a book called The Lost World of Dune. The first edition was the first time a "magic book" was published. It was a sequel to the original Dungeon Master's Guide. In its early years, it was not a game, but a collection of rules for a world that was full of monsters and dungeons. Since its release, the Dungeon Player's Handbook has been heavily revised. This book is the definitive guide to Dungeons & Dragons. Every character is unique and every dungeon is full in its own right.

The Dungeon Guide is an extremely comprehensive guide, giving you everything you need to master the game. There are a lot of great tips and strategies, as well as some useful tricks. A very good starting point for any new DMs!
. And, if you're looking for more 

In [18]:
# Generate and format the story
story = generate_story(prompt, model, tokenizer, device)
formatted_story = clean_story(story, max_line_length=80)  # Clean and format the story with wrapping
print(formatted_story)



Once upon a time, in a magical forest, a young prince discovered a hidden
treasure... He was called the Princess of the Forest, and she was the daughter
of a noble, an immortal, the father of many, of countless people. The land of
his birth was ruled by a king, who ruled over all his subjects, which would have
meant that he was his own child.

This was what had happened to him, he knew that the land was cursed, that it was
not worth it. . When the prince was a child, his father had given him a son
named, named his son, to take care of.

In this case, it would not have been the same as the one he had taken care not
to give his child to. However, as a boy, this was something that was difficult
for him to handle. He had to learn how to be a warrior, how not even his
ancestors could do this.

After all, there was only one noble who could take this caretaking child away
from him... The King of Ferelden. Not only that, but the king of this land had
also decided to send him away as well.

It 

In [22]:
prompt = "A robot named Max in a desert..."

# Generate the story
story = generate_story(prompt, model, tokenizer, device)

formatted_story = clean_story(story, max_line_length=80)  # Clean and format the story with wrapping
print(formatted_story)

A robot named Max in a desert... But that's not what happened. It was a robot
called Max that had been stolen from the warehouse and taken to the nearest
town.

"The city was attacked by bandits, and the robot was destroyed." ...What did Max
do? "It was saved by an elderly man. He had left the town for some reason." The
robot finally had the time to figure out what was going on, but he was already
at the center of the fight.

That man was Max. In the end, Max made Max's life a whole lot easier. "Max was
the one who saved the city and saved its citizens." "We were the ones that saved
Max, so we have to do something for him." Max was in his late teens.

Max had already grown up around magic, he had learned how to use it, how it
could be used to make people happy. But he still felt that he couldn't go back
to school, because he would be an orphan. At the same time, it was not like he
could do anything about it.

Instead, his childhood was filled with troubles. This was why he found out th